<a href="https://colab.research.google.com/github/chetools/CHE4061_Spring2026/blob/main/NRTL_VLEFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
%run che5.ipynb

In [2]:
p=Props(['Isopropanol','Water','Toluene'])

In [3]:
p.NRTL_gamma(np.array([0.3,0.3,0.4]), 300.)

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [4]:
#NRTL Parameters: Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T
def NRTL_Gex(x, T):
    tau = p.NRTL_A+p.NRTL_B/T + p.NRTL_C*jnp.log(T)+p.NRTL_D*T
    G = jnp.exp(-p.NRTL_alpha*tau)
    tauG = tau*G
    tauGx = jnp.einsum('ji,j->i', tauG, x)
    Gx = jnp.einsum('ki,k->i', G, x)
    return jnp.sum(x*tauGx/Gx)

In [5]:
Gex_grad = jax.grad(NRTL_Gex, 0)

In [6]:
x=np.array([0.3,0.3,0.4])
T=300.
jnp.exp(Gex_grad(x,T))

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [7]:
def NRTL(x, T):
    tau = p.NRTL_A+p.NRTL_B/T + p.NRTL_C*jnp.log(T)+p.NRTL_D*T
    G = jnp.exp(-p.NRTL_alpha*tau)
    tauG = tau*G
    xtauG = jnp.einsum('ji,j->i', tauG, x)
    xG = jnp.einsum('ki,k->i', G, x)
    xtauGdivxG = xtauG/xG
    return jnp.exp(xtauGdivxG + jnp.einsum('j,ij->i',x,G*(tau - xtauGdivxG[None,:])/xG))


In [8]:
NRTL(x,T)

Array([1.12930337, 6.26607585, 3.13419067], dtype=float64)

In [9]:
r=Props(['Isopropanol', 'Water'])
z = np.array([0.3,0.7])

In [10]:
def bubbleP_NRTL(x,T):
    yP = x*r.NRTL_gamma(x,T)* r.Pvap(T)
    P = np.sum(yP)
    return P, yP/P

In [11]:
bubbleP_NRTL(z,300.)

(Array(7416.3415318, dtype=float64),
 Array([0.54601004, 0.45398996], dtype=float64))

In [12]:
def dewP_NRTL(y, T, Niter = 100, tol=1e-6):
    Pvap = r.Pvap(T)

    g = np.ones_like(y)
    for i in range(Niter):
        Pold = 1/np.sum(y/(g*Pvap))
        x = y*Pold/(g*Pvap)
        x = x/np.sum(x)
        g = r.NRTL_gamma(x,T)
        Pnew = 1/np.sum(y/(g*Pvap))
        if np.abs(Pnew-Pold)/Pnew<tol:
            break

    x = y*Pnew/(g*Pvap)
    x = x/np.sum(x)
    return Pnew, x, i

In [13]:
dewP_NRTL(z, 300.)

(Array(5002.32678518, dtype=float64),
 Array([0.01592674, 0.98407326], dtype=float64),
 6)

In [14]:
T=350.
x1s = np.linspace(0,1,101)
y1s = []
bubblePs = []
for x1 in x1s:
    P, (y1, _) = bubbleP_NRTL(np.array([x1, 1-x1]), T)
    bubblePs.append(P)
    y1s.append(y1)
bubblePs = np.array(bubblePs)
y1s = np.array(y1s)

In [15]:
fig = make_subplots()
fig.add_scatter(x=x1s, y=bubblePs, name='bubble')
fig.add_scatter(x=y1s, y=bubblePs, name='dew')
fig.update_layout(width=600, height=400)

In [16]:
def bubbleT_NRTL(x, P):

    T0 = np.sum(x*r.Tb(P))
    def froot(T):
        return np.sum(x*r.NRTL_gamma(x,T)*r.Pvap(T)/P)-1.

    T=sp.optimize.root_scalar(froot, x0=T0, method='secant').root
    return T, x*r.NRTL_gamma(x,T)*r.Pvap(T)/P

In [24]:
#Ideal dewT calculation for initial guess to dewT_NRTL

def dewP(y, T):
    Pvap = r.Pvap(T)
    P = 1/np.sum(y/Pvap)
    return P, y*P/Pvap

def dewT(y, P):

    T0 = np.sum(y*r.Tb(P))
    def froot(T):
        return dewP(y,T)[0]-P

    T=sp.optimize.root_scalar(froot, x0=T0, method='secant').root
    return T, y*P/r.Pvap(T)

In [30]:
#$100 problem
#x=[0.2, 0.3] and last x = 0.5
#in search of v, such that exp(v)/np.sum(exp(v)) = [0.2, 0.3]

In [35]:
def dewT_NRTL(y, P):
    T0, x0 = dewT(y,P)
    v0 = np.r_[T0, x0]

    def froot(v):
        T=v[0]
        x=v[1:]
        return np.r_[x*r.NRTL_gamma(x,T)*r.Pvap(T)/P-y, np.sum(x)-1]

    res=sp.optimize.root(froot, v0).x

    return res[0], res[1:]




In [36]:
dewT_NRTL(np.array([0.5,0.5]), 1e5)

(np.float64(356.099138117727), array([0.10205311, 0.89794689]))

In [17]:
P=1e5
x1s = np.linspace(0,1,101)
y1s = []
bubbleTs = []
for x1 in x1s:
    T, (y1, _) = bubbleT_NRTL(np.array([x1, 1-x1]), P)
    bubbleTs.append(T)
    y1s.append(y1)
bubbleTs = np.array(bubbleTs)
y1s = np.array(y1s)

In [18]:
fig = make_subplots(rows=1,cols=2)
fig.add_scatter(x=x1s, y=bubbleTs, name='bubble', row=1,col=1)
fig.add_scatter(x=y1s, y=bubbleTs, name='dew', row=1,col=1)
fig.add_scatter(x=x1s, y=y1s, row=1, col=2, showlegend=False)
fig.add_scatter(x=[0,1],y=[0,1], row=1, col=2, mode='lines', showlegend=False, line_color='grey')
fig.update_layout(width=800, height=400)

In [ ]:
def flashPT(z, P, T):
    dP, _ = dewP(z, T)
    bP, _ = bubbleP(z, T)
    if P>bP:
        return 'All Liquid'
    if P<dP:
        return 'All Gas'

    phi_guess = (bP - P)/(bP-dP)

    K=p.Pvap(T)/P

    def froot(phi):
        return jnp.sum(z*(K-1)/(1+phi*(K-1)))

    froot_prime= jax.grad(froot)

    froot = jax.jit(froot)

    phi=sp.optimize.root_scalar(froot, x0=phi_guess, method='Newton', fprime = froot_prime).root
    x = z/(1+phi*(K-1))
    y = K*x
    return phi, x, y

In [ ]:
flashPT(np.array([0.4,0.6]), 1e5, 370.)

In [ ]:
def flashPT_NRTL(z, P, T):
    dP, dew_x = dewP_NRTL(z, T)
    bP, bubble_y = bubbleP_NRTL(z, T)
    if P>bP:
        return 'All Liquid'
    if P<dP:
        return 'All Gas'

    phi_guess = (bP - P)/(bP-dP)


    K=r.NRTL_gamma(x,T )*r.Pvap(T)/P

    def froot(phi):
        return jnp.sum(z*(K-1)/(1+phi*(K-1)))

    froot_prime= jax.grad(froot)

    froot = jax.jit(froot)

    phi=sp.optimize.root_scalar(froot, x0=phi_guess, method='Newton', fprime = froot_prime).root
    x = z/(1+phi*(K-1))
    y = K*x
    return phi, x, y